In [13]:
import pandas as pd
df = pd.read_csv('ptask (1)/data_1_23_40_66.dat')

In [15]:
with open('ptask (1)/data_1_23_40_66.dat', 'r') as file:
    lines = file.readlines()


In [16]:
def parse_dat_file(filepath):
    with open(filepath, "r") as f:
        lines = [line.strip() for line in f if line.strip() and not line.startswith("#")]

    data = {}
    i = 0

    # Parse scalar key-value pairs like "Type = 1"
    while i < len(lines):
        if "=" in lines[i]:
            key, val = lines[i].split("=")
            key, val = key.strip(), val.strip()

            if key == "Jobs":
                num_jobs = int(val)
                data["Jobs"] = []
                i += 1
                # Parse job lines (each has 2 integers)
                for _ in range(num_jobs):
                    job = list(map(int, lines[i].split()))
                    data["Jobs"].append(job)
                    i += 1
            elif key == "Qualifications":
                num_quals = int(val)
                data["Qualifications"] = []
                i += 1
                for _ in range(num_quals):
                    # Format: "28:  30 36 1 2 3..."
                    colon_idx = lines[i].index(":")
                    skill_line = list(map(int, lines[i][colon_idx + 1:].split()))
                    data["Qualifications"].append(skill_line)
                    i += 1
            else:
                data[key] = int(val)
                i += 1
        else:
            i += 1  # Skip any junk lines (just in case)

    return data


In [17]:
data = parse_dat_file("ptask (1)/data_1_23_40_66.dat")
print(f"Type: {data['Type']}")
print(len(data["Qualifications"]), "employees with skill sets")

Type: 1
23 employees with skill sets


In [18]:
with open("ptask (1)/data_1_23_40_66.dat", "r") as file:
    print(file.read())

# Randomly generated data for apersonnel scheduling problem
# ./datagen tightness = 90 Multi-skilling level = 66
# Random number generator seed = 0
Type = 1
Jobs = 40
  43  516
 164  746
  75  591
 230  718
 839 1354
  96  637
   1  593
 179  713
 130  765
 119  688
 194  783
 270  818
 102  618
 774 1291
  28  550
  56  630
 758 1350
 804 1320
  16  557
  16  565
  30  536
 186  752
 677 1259
 739 1244
 834 1313
 724 1346
 761 1304
 823 1396
 569 1040
 804 1340
 125  740
 764 1256
 159  656
 712 1278
 726 1291
 651 1235
 750 1278
 725 1363
 867 1376
 844 1319
Qualifications = 23
 26:   6  13   0   1   2   3   4   8  11  12  14  15  16  17  20  21  23  24  25  26  29  30  31  33  37  39 
 24:  14  16   1   4   6   8  12  15  17  18  19  20  22  23  25  27  28  31  34  35  36  37  38  39 
 28:  30  36   1   2   3   4   6   9  10  11  12  13  15  16  17  18  21  22  23  24  26  27  32  33  34  35  38  39 
 24:   7  25   0   2   4   5   6  10  13  15  16  17  21  23  26  27  29  30  31  3

In [11]:
import random

def initialize_population(num_individuals, num_jobs, qualifications):
    """
    Initialize a population for the scheduling problem.
    
    Parameters:
        - num_individuals: The number of individuals (solutions) in the population.
        - num_jobs: The total number of jobs.
        - qualifications: The qualification list mapping workers to jobs.
        
    Returns:
        - population: A list of individuals representing the schedule.
    """
    population = []
    
    for _ in range(num_individuals):
        individual = []
        
        for job_idx in range(num_jobs):
            qualified_workers = qualifications.get(job_idx, [])
            if qualified_workers:
                # Randomly assign a qualified worker to the job
                worker = random.choice(qualified_workers)
                individual.append(worker)
            else:
                # No qualified worker for this job, handle accordingly
                individual.append(None)
        
        population.append(individual)
    
    return population


In [12]:
num_individuals = 10 
num_jobs = data["Jobs"]
qualifications = data["Qualifications"]

initial_population = initialize_population(num_individuals, num_jobs, qualifications)
print(initial_population[:5])

TypeError: 'list' object cannot be interpreted as an integer

In [ ]:
from deap import base, creator, tools, algorithms
# step 1: create problem definition
creator.create("FitnessMulti", base.Fitness, weights=(-1.0, -1.0))
creator.create("Individual", list, fitness=creator.FitnessMulti)

# step 2: create the toolbox
toolbox = base.Toolbox()
toolbox.register("individual", generate_individual) # need to make generate_individual
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate_schedule) # need to make evaluate_schedule
toolbox.register("mate", crossover_schedules)
toolbox.register("mutate", mutate_schedule)
toolbox.register("select", tools.selNSGA2)

#step 3: run
population = toolbox.population(n=100)
algorithms.eaMuPlusLambda(population, toolbox, mu=100, lambda_=200, cxpb=0.7, mutpb=0.3, ngen=100)